In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(palette='muted',style='whitegrid')
from sklearn.manifold import TSNE

In [2]:
def add_feature_no_zeros():
    """
    Add feature which tells the no. of occurences of zeros across a datapoint
    """
    for df in [train,test,val]:
        df['no_zeros'] = df

ModuleNotFoundError: No module named 'EDA'

In [3]:
train = pd.read_pickle('pickle_files/train_1.pkl')
test = pd.read_pickle('pickle_files/test_1.pkl')

In [23]:
(test==-1).sum().sum()

0